# 데이터 전처리

0. 데이터 확인하기  
1. 결측치(Missing Data)  
2. 중복된 데이터  
3. 정규화(Normalization)  
4. 이상치(Outlier)  
5. 원-핫 인코딩(One-Hot Encoding)  


In [ ]:
import pandas as pd
import numpy as np


인천교통공사_수송인원통계(1,2호선 역별시간대별 이용 인원현황_20191231.csv  
역별시간대별 이용인원현황(역사별, 시간대별, 승하차 인원)   
[(데이터 출처)](https://www.data.go.kr/data/15004329/fileData.do)



In [ ]:
import os
os.chdir('/content/drive/My Drive/_share/_2020/data')

In [ ]:
metro = pd.read_csv('인천교통공사_수송인원통계(1,2호선 역별시간대별 이용 인원현황_20191231_수정.csv')
metro.head()

In [ ]:
metro.describe()

In [ ]:
metro.describe(include=object)

',' 때문에 숫자가 아닌 데이터 타입으로 들어간 것을 확인할 수 있음.  
아래 cols 컬럼의 데이터들은 ','를 ''로 바꾸고, 데이터 확인


In [ ]:
cols = ['06~07', '07~08', '08~09', '09~10', '10~11',
       '11~12', '12~13', '13~14', '14~15', '15~16', '16~17', '17~18', '18~19',
       '19~20', '20~21', '21~22', '22~23', '23~24']

In [ ]:
metro_cols = metro[cols].applymap(lambda x: float(str(x).replace(',', '')))
metro_cols.head()

In [ ]:
metro[cols] = metro_cols
metro.head()

In [ ]:
metro.tail()

In [ ]:
metro.describe(include=object)

In [ ]:
metro.describe()

## step1. 결측치(Missing Data) 


In [ ]:
print('전체 데이터 건수:', len(metro))

전체 데이터 건수에서 각 컬럼별 값이 있는 데이터 수를 빼주면 컬럼별 결측치의 개수를 알 수 있음

In [ ]:
print('컬럼별 결측치 개수')
len(metro) - metro.count()

### Q. 컬럼별 결측치 비율을 계산해보기
위 코드를 이용하여 개수가 아닌 비율을 계산해보자

In [ ]:
print('컬럼별 결측치 비율')
(len(metro) - metro.count())/len(metro)

‘기타사항’을 보시면 전부 결측치라는 것을 알 수 있음.  


### Q. 기타사항 컬럼 삭제하기

In [ ]:
metro = metro.drop('기타사항', axis=1)
metro.head()

row에 하나라도 null값이 있는 데이터 확인하기

In [ ]:
metro[metro.isnull().any(axis=1)]

### Q. 행 삭제하기
대부분의 데이터가 없는 40881 행을 삭제해보기

In [ ]:
metro.dropna(subset=['역명'], inplace=True)

In [ ]:
metro[metro.isnull().any(axis=1)]

### Q. index 40266의 07~08을 채워넣어보기. 
(어떤 값으로 채울지는 각자 생각해보기)

In [ ]:
idx = 40266 
col = '07~08'
station = '주안'
flag = '승차'

In [ ]:
idx1 = metro[(metro['통행일자']=='2019.12.13')&(metro['역명']==station)&(metro['구분']==flag)].index
idx2 = metro[(metro['통행일자']=='2019.12.27')&(metro['역명']==station)&(metro['구분']==flag)].index

metro.loc[idx, col] = (metro.loc[idx1, col].values + metro.loc[idx2, col].values)/2
metro.loc[[idx]]

## step2. 중복된 데이터

중복된 데이터가 있는지 확인하기

In [ ]:
metro[metro.duplicated()]

In [ ]:
metro[metro.duplicated(subset=['호선', '통행일자', '역명', '구분'])]

In [ ]:
metro[(metro['역명']=='아시아드경기장') & (metro['통행일자']=='2019.12.31')&(metro['구분']=='하차')]

In [ ]:
len(metro)

### Q. 중복된 데이터 삭제하기 


In [ ]:
metro.drop_duplicates(subset=['호선', '통행일자', '역명', '구분'], keep='first', inplace=True)

In [ ]:
len(metro)

## step3. 이상치(Outlier)

metro_bp : 부평역의 데이터

In [ ]:
metro_bp = metro.groupby('역명').get_group('부평')
metro_bp.head()

In [ ]:
def outlier(df, col, z):
    return df[abs(df[col] - np.mean(df[col]))/np.std(df[col])>z].index

In [ ]:
metro_bp.loc[outlier(metro_bp, '18~19', 1.5)]

In [ ]:
metro_bp.loc[outlier(metro_bp, '18~19', 3)]

### Q. 이상치가 아닌 데이터
이상치가 아닌 데이터의 인덱스만 추출하기 위한 함수 not_outlier를 작성해보기  
위 코드 참고하기

In [ ]:
def not_outlier(df, col, z):
    return df[abs(df[col] - np.mean(df[col]))/np.std(df[col]) <= z].index

In [ ]:
metro_bp = metro_bp.loc[not_outlier(metro_bp, '18~19', 3)]
metro_bp

## step4. 정규화(Normalization)

cols : 수치형 데이터 중 '호선'을 제외한 컬럼 

In [ ]:
cols = metro.select_dtypes([np.number]).columns
cols

In [ ]:
cols = cols.drop('호선')
cols

||||
|---|---|---|
|Standardization|데이터의 평균은 0, 분산은 1로 변환|$${\frac {X-\mu }{\sigma }}$$|
|Min-Max Scaling|데이터의 최솟값은 0, 최댓값은 1로 변환|$${{\frac {X-X_{\min }}{X_{\max }-X_{\min }}}}$$|

### Q 데이터에서 평균을 빼고, 표준편차로 나눠주기
**Standardization**  


In [ ]:
metro_bp_Standardization= (metro_bp[cols]-metro_bp[cols].mean())/metro_bp[cols].std()
metro_bp_Standardization.head()

각 컬럼의 평균들을 보면 0에 가깝고, 표준편차는 1에 가까운 것을 확인

In [ ]:
metro_bp_Standardization.describe()

### Q 데이터에서 최솟값을 빼주고, '최댓값-최솟값'으로 나눠주기
**Min-Max Scaling**  


In [ ]:
metro_bp[cols] = (metro_bp[cols]-metro_bp[cols].min())/(metro_bp[cols].max()-metro_bp[cols].min())
metro_bp.head()

Min-Max Scaling 방법으로 정규화시킨 후, 각 컬럼의 최솟값(min)은 0이고, 최댓값(max)은 1임을 확인할 수 있습니다. 

In [ ]:
metro_bp.describe()

**주의!!**  
train 데이터와 test 데이터가 나눠져 있는 경우 train 데이터를 정규화 시켰던 기준으로 test 데이터도 정규화 시켜줘야 함. 

In [ ]:
train = pd.DataFrame([[10, -10], [30, 10], [50, 0]])
test = pd.DataFrame([[0, 1], [10, 10]])

In [ ]:
train_min = train.min()
train_max = train.max()

In [ ]:
train_min

In [ ]:
train_min_max = (train - train_min)/(train_max - train_min)
test_min_max =  (test - train_min)/(train_max - train_min)

In [ ]:
train_min_max

In [ ]:
test_min_max

## step5. 원-핫 인코딩(One-Hot Encoding)

이제 범주형 데이터인 국가명 컬럼을 다뤄보도록 하겠습니다.  

머신러닝이나 딥러닝 프레임워크에서 범주형을 지원하지 않는 경우 원-핫 인코딩을 해야 합니다.  

여러분은 지난 시간에 포켓몬 데이터를 다루면서 원-핫 인코딩에 대한 개념을 익히셨습니다. 기억나시나요?

원-핫 인코딩이란 무엇인가요?

원-핫 인코딩이란 카테고리별 이진 특성을 만들어 해당하는 특성만 1 나머지는 0으로 만드는 방법입니다.는

그럼 pandas로 국가명 컬럼을 원-핫 인코딩을 해보겠습니다. 

pandas에서 `get_dummies` 함수를 통해 손쉽게 원-핫 인코딩을 할 수 있습니다. 


In [ ]:
metro_bp_dm = pd.get_dummies(metro_bp['구분'])
metro_bp_dm.head()

`concat` 함수로 데이터프레임 trade와 countury를 합쳐줍니다.  

In [ ]:
metro_bp = pd.concat([metro_bp, metro_bp_dm], axis=1)
metro_bp.head()

이제는 필요 없어진 국가명 컬럼을 삭제해주고 나면 trade는 우리가 원하는 데이터프레임이 됩니다. 

In [ ]:
metro_bp.drop(['구분','호선','역명'], axis=1, inplace=True)
metro_bp.head()